In [1]:
!pip install openrouteservice

In [2]:
import numpy as np

yangon_shops = [
    [96.1550, 16.7800],
    [96.1500, 16.7990],
    [96.1750, 16.8415],
    [96.1300, 16.8510],
    [96.1380, 16.8330],
    [96.1455, 16.7840],
    [96.1700, 16.8070],
    [96.1620, 16.7750],
    [96.1290, 16.8700],
    [96.1975, 16.8580],
]

print(yangon_shops)


[[96.155, 16.78], [96.15, 16.799], [96.175, 16.8415], [96.13, 16.851], [96.138, 16.833], [96.1455, 16.784], [96.17, 16.807], [96.162, 16.775], [96.129, 16.87], [96.1975, 16.858]]


In [3]:
from google.colab import userdata
api_key = userdata.get('OPEN_ROUTE_SERVICE_KEY')

In [4]:
import openrouteservice as ors
import folium

# Initialize the OpenRouteService client
client = ors.Client(key=api_key)

# Modify the vehicle start location to ensure it's within a valid road network (for example, Beijing)
vehicle_start = [96.15, 16.8]

# Define vehicles for optimization
vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[6]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[5]),
]

# Define jobs with the coordinates
jobs = [
    ors.optimization.Job(id=index, location=coords, amount=[1])
    for index, coords in enumerate(yangon_shops)
]

# Request for optimization
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)


In [6]:
# Create a map centered around the average coordinates
average_lat = sum([coord[1] for coord in yangon_shops]) / len(yangon_shops)
average_lon = sum([coord[0] for coord in yangon_shops]) / len(yangon_shops)
m = folium.Map(location=[average_lat, average_lon], tiles="cartodbpositron", zoom_start=5)
m

In [7]:
# Add markers for all coordinates
for coord in yangon_shops:
    folium.Marker(location=[coord[1], coord[0]]).add_to(m)  # Add markers for each job

# Add vehicle start location marker (red icon)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], icon=folium.Icon(color="red")).add_to(m)

# Add optimized routes to the map
line_colors = ['green', 'orange', 'blue', 'yellow']
for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(m)

# Display the map
m